In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import math
from statistics import mean
import requests 
import xlsxwriter

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_IEX import IEX_CLOUD_API_TOKEN

In [4]:
stocks.drop(135,inplace=True)
stocks.drop(219,inplace=True)
stocks.drop(467,inplace=True)
stocks.drop(484,inplace=True)

In [5]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [6]:
lst=[]
for i in range(6):
    lst.append(stocks['Ticker'][i*100:(i+1)*100])

In [7]:
query_string=[]
for i in lst:
    query_string.append(','.join(i))

In [28]:
columns = [
    'Ticker',
    'Price',
    'Gross Profit',
    'Price to Earnings Ratio',
    'PE Percentile',
    'Price to Book Ratio',
    'PB Percentile',
    'Price to Sales Ratio',
    'PS Percentile',
    'EV to EBITDA',
    'EV to EBITDA Percentile',
    'EV to Gross Profit',
    'EV to Gross Profit Percentile',
    'RV Score',    
    'Number of Shares to Buy', 
]


In [29]:
value_df= pd.DataFrame(columns=columns)

In [30]:
for query in query_string:
    batch_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={query}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_url).json()
    for ticker in query.split(','):
        enterprise_value = data[ticker]['advanced-stats']['enterpriseValue']
        ebitda = data[ticker]['advanced-stats']['EBITDA']
        gross_profit = data[ticker]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        value_df = value_df.append(pd.Series([ticker,
                                            data[ticker]['quote']['latestPrice'],
                                            data[ticker]['advanced-stats']['grossProfit'],
                                            data[ticker]['quote']['peRatio'],
                                            'N/A',
                                            data[ticker]['advanced-stats']['priceToBook'],
                                            'N/A',
                                            data[ticker]['advanced-stats']['priceToSales'],
                                            'N/A',
                                            ev_to_ebitda,
                                            'N/A',
                                            ev_to_gross_profit,
                                            'N/A',
                                            'N/A',
                                              'N/A'], 
                                            index = columns), 
                                            ignore_index = True)

In [31]:
value_df

,Ticker,Price,Gross Profit,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV to EBITDA,EV to EBITDA Percentile,EV to Gross Profit,EV to Gross Profit Percentile,RV Score,Number of Shares to Buy
0,A,127.100,3545072999,30.28,N/A,7.36,N/A,5.72,N/A,21.450979,N/A,11.257780,N/A,N/A,N/A
1,AAL,14.470,36044443645,-4,N/A,-1.0202,N/A,0.2545,N/A,-21.461329,N/A,0.977814,N/A,N/A,N/A
2,AAP,184.720,5085103413,21.01,N/A,3.9,N/A,1.0266,N/A,11.563103,N/A,2.481723,N/A,N/A,N/A
3,AAPL,149.180,175038941994,24.58,N/A,35.41,N/A,6.11,N/A,18.675845,N/A,13.977979,N/A,N/A,N/A
4,ABBV,157.430,40963018657,22.09,N/A,17.38,N/A,4.83,N/A,11.621210,N/A,8.468413,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.220,4954527038,21.98,N/A,-3.91,N/A,5,N/A,19.555996,N/A,9.268730,N/A,N/A,N/A
497,ZBH,103.650,5328755303,105,N/A,1.9,N/A,2.94,N/A,17.444205,N/A,5.339795,N/A,N/A,N/A
498,ZBRA,308.100,2715881251,19.74,N/A,5.43,N/A,2.7,N/A,15.054036,N/A,6.048860,N/A,N/A,N/A
499,ZION,51.579,3775480441,5.44,N/A,1.3,N/A,2.26,N/A,4.046734,N/A,2.034537,N/A,N/A,N/A


In [32]:
clm=['Gross Profit','Price to Earnings Ratio', 'Price to Book Ratio','Price to Sales Ratio',  'EV to EBITDA','EV to Gross Profit']

In [33]:
value_df.dropna(subset=clm,inplace=True)

In [34]:
value_df

,Ticker,Price,Gross Profit,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV to EBITDA,EV to EBITDA Percentile,EV to Gross Profit,EV to Gross Profit Percentile,RV Score,Number of Shares to Buy
0,A,127.100,3545072999,30.28,N/A,7.36,N/A,5.72,N/A,21.450979,N/A,11.257780,N/A,N/A,N/A
1,AAL,14.470,36044443645,-4,N/A,-1.0202,N/A,0.2545,N/A,-21.461329,N/A,0.977814,N/A,N/A,N/A
2,AAP,184.720,5085103413,21.01,N/A,3.9,N/A,1.0266,N/A,11.563103,N/A,2.481723,N/A,N/A,N/A
3,AAPL,149.180,175038941994,24.58,N/A,35.41,N/A,6.11,N/A,18.675845,N/A,13.977979,N/A,N/A,N/A
4,ABBV,157.430,40963018657,22.09,N/A,17.38,N/A,4.83,N/A,11.621210,N/A,8.468413,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.220,4954527038,21.98,N/A,-3.91,N/A,5,N/A,19.555996,N/A,9.268730,N/A,N/A,N/A
497,ZBH,103.650,5328755303,105,N/A,1.9,N/A,2.94,N/A,17.444205,N/A,5.339795,N/A,N/A,N/A
498,ZBRA,308.100,2715881251,19.74,N/A,5.43,N/A,2.7,N/A,15.054036,N/A,6.048860,N/A,N/A,N/A
499,ZION,51.579,3775480441,5.44,N/A,1.3,N/A,2.26,N/A,4.046734,N/A,2.034537,N/A,N/A,N/A


In [35]:
metrics = {
            'Price to Earnings Ratio': 'PE Percentile',
            'Price to Book Ratio':'PB Percentile',
            'Price to Sales Ratio': 'PS Percentile',
            'EV to EBITDA':'EV to EBITDA Percentile',
            'EV to Gross Profit':'EV to Gross Profit Percentile'
}

In [36]:
for index in value_df.index:
    for metric in metrics.keys():
        value_df.loc[index, metrics[metric]] = stats.percentileofscore(value_df[metric], value_df.loc[index, metric])


In [38]:
for index in value_df.index:
    percentiles = []
    for metric in metrics.keys():
        percentiles.append(value_df.loc[index, metrics[metric]])
    value_df.loc[index, 'RV Score'] = mean(percentiles)

In [39]:
value_df

,Ticker,Price,Gross Profit,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV to EBITDA,EV to EBITDA Percentile,EV to Gross Profit,EV to Gross Profit Percentile,RV Score,Number of Shares to Buy
0,A,127.100,3545072999,30.28,72.8216,7.36,81.5353,5.72,81.9502,21.450979,81.1203,11.257780,81.1203,79.7095,N/A
1,AAL,14.470,36044443645,-4,3.3195,-1.0202,6.22407,0.2545,1.65975,-21.461329,0.829876,0.977814,3.94191,3.19502,N/A
2,AAP,184.720,5085103413,21.01,51.4523,3.9,60.4772,1.0266,16.5975,11.563103,41.2863,2.481723,15.5602,37.0747,N/A
3,AAPL,149.180,175038941994,24.58,60.7884,35.41,96.888,6.11,84.0249,18.675845,72.6141,13.977979,88.1743,80.4979,N/A
4,ABBV,157.430,40963018657,22.09,54.4606,17.38,93.1535,4.83,76.6598,11.621210,42.1162,8.468413,66.805,66.639,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.220,4954527038,21.98,53.9419,-3.91,5.80913,5,78.6307,19.555996,76.1411,9.268730,72.6141,57.4274,N/A
497,ZBH,103.650,5328755303,105,98.1328,1.9,29.4606,2.94,55.6017,17.444205,69.0871,5.339795,42.3237,58.9212,N/A
498,ZBRA,308.100,2715881251,19.74,48.9627,5.43,72.3029,2.7,51.2448,15.054036,60.3734,6.048860,47.7178,56.1203,N/A
499,ZION,51.579,3775480441,5.44,7.67635,1.3,17.1162,2.26,43.6722,4.046734,6.43154,2.034537,12.6556,17.5104,N/A


In [40]:
value_df.sort_values(by='RV Score',inplace= True)
Top50 = value_df[:50]

In [42]:
Top50.reset_index(drop = True, inplace = True)

In [43]:
Top50

,Ticker,Price,Gross Profit,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV to EBITDA,EV to EBITDA Percentile,EV to Gross Profit,EV to Gross Profit Percentile,RV Score,Number of Shares to Buy
0,AAL,14.470,36044443645,-4,3.3195,-1.0202,6.22407,0.2545,1.65975,-21.461329,0.829876,0.977814,3.94191,3.19502,N/A
1,KSS,30.300,8036222282,4.6,6.0166,0.8523,9.12863,0.1982,0.829876,3.410687,4.14938,0.999262,5.18672,5.06224,N/A
2,PVH,57.770,5459875470,4.29,5.60166,0.7467,8.29876,0.4207,3.94191,4.248320,7.46888,0.978449,4.14938,5.89212,N/A
3,GM,32.430,25981207046,5.51,8.09129,0.735,7.88382,0.3596,3.3195,1.644390,2.07469,1.412417,8.09129,5.89212,N/A
4,AIG,52.890,53601627374,4.75,6.22407,0.7436,8.09129,0.7695,12.0332,2.721613,3.11203,0.781110,1.86722,6.26556,N/A
5,GPS,8.520,6117615598,-44.71,1.03734,1.35,18.0498,0.2086,1.03734,4.760048,10.166,0.720475,1.65975,6.39004,N/A
6,LNC,48.090,20365570392,7.04,11.4108,0.5693,7.05394,0.4309,4.77178,4.905146,10.7884,0.408659,0.207469,6.84647,N/A
7,UNM,34.745,12092433747,7.82,13.278,0.6629,7.67635,0.5748,7.67635,3.845203,5.18672,0.584337,0.829876,6.92946,N/A
8,F,11.950,20947163861,4.06,5.18672,1.0099,12.0332,0.333,2.48963,3.554358,4.97925,1.750247,10.9959,7.13693,N/A
9,SYF,31.194,16791983942,4.15,5.39419,1.18,14.2116,0.9084,13.9004,2.459293,2.48963,0.893065,2.90456,7.78008,N/A


In [44]:
while True:
    try:
        portfolio= float(input('Enter the size of the Portfolio in USD '))
        break
    except ValueError:
        print("Please enter a Number")

Enter the size of the Portfolio in USD 1000000


In [45]:
position_size=portfolio/len(Top50.index)

In [46]:
for i in Top50.index:
    Top50.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Top50['Price'][i])

C:\Users\Vighnesh Deshpande\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [47]:
Top50

,Ticker,Price,Gross Profit,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV to EBITDA,EV to EBITDA Percentile,EV to Gross Profit,EV to Gross Profit Percentile,RV Score,Number of Shares to Buy
0,AAL,14.470,36044443645,-4,3.3195,-1.0202,6.22407,0.2545,1.65975,-21.461329,0.829876,0.977814,3.94191,3.19502,1382
1,KSS,30.300,8036222282,4.6,6.0166,0.8523,9.12863,0.1982,0.829876,3.410687,4.14938,0.999262,5.18672,5.06224,660
2,PVH,57.770,5459875470,4.29,5.60166,0.7467,8.29876,0.4207,3.94191,4.248320,7.46888,0.978449,4.14938,5.89212,346
3,GM,32.430,25981207046,5.51,8.09129,0.735,7.88382,0.3596,3.3195,1.644390,2.07469,1.412417,8.09129,5.89212,616
4,AIG,52.890,53601627374,4.75,6.22407,0.7436,8.09129,0.7695,12.0332,2.721613,3.11203,0.781110,1.86722,6.26556,378
5,GPS,8.520,6117615598,-44.71,1.03734,1.35,18.0498,0.2086,1.03734,4.760048,10.166,0.720475,1.65975,6.39004,2347
6,LNC,48.090,20365570392,7.04,11.4108,0.5693,7.05394,0.4309,4.77178,4.905146,10.7884,0.408659,0.207469,6.84647,415
7,UNM,34.745,12092433747,7.82,13.278,0.6629,7.67635,0.5748,7.67635,3.845203,5.18672,0.584337,0.829876,6.92946,575
8,F,11.950,20947163861,4.06,5.18672,1.0099,12.0332,0.333,2.48963,3.554358,4.97925,1.750247,10.9959,7.13693,1673
9,SYF,31.194,16791983942,4.15,5.39419,1.18,14.2116,0.9084,13.9004,2.459293,2.48963,0.893065,2.90456,7.78008,641


In [63]:
writer = pd.ExcelWriter('Quantitative_Value_Strategy.xlsx', engine='xlsxwriter')
Top50.to_excel(writer, sheet_name='Quantitative Value Strategy', index = False)

In [64]:
background_color = '#ffffff'
font_color = '#000000'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_format = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [65]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Gross Profit', dollar_format],
                    'D': ['Price to Earnings Ratio', float_format],
                    'E': ['PE Percentile', percent_format],
                    'F': ['Price to Book Ratio', float_format],
                    'G': ['PB Percentile',percent_format],
                    'H': ['Price to Sales Ratio', float_format],
                    'I': ['PS Percentile', percent_format],
                    'J': ['EV to EBITDA', float_format],
                    'K': ['EV to EBITDA Percentile', percent_format],
                    'L': ['EV to Gross Product', float_format],
                    'M': ['EVto Gross Product Percentile', percent_format],
                    'N': ['RV Score', percent_format],
                    'O': ['Number of Shares to Buy', integer_format]
                 }

for column in column_formats.keys():
    writer.sheets['Quantitative Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Quantitative Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [66]:
writer.save()